# Calculate Embeddings

Ajuda a calcular embeddings dos dados adquiridos dos comentários das redes sociais de políticos. (Planilha de Gabrielly)

Entrada: `LulaTotal Validado.xlsx` e `Bolsonaro Validado.xlsx`

Saída: `Embeddings_(NOME_DO_MODELO)_(REDE_SOCIAL).xlsx`

In [99]:
import pandas as pd
from sentence_transformers import SentenceTransformer

BASE_PATH = 'dados/'
DIRECTORY = BASE_PATH + 'preprocessed/embeddings/'
SOCIAL_NETWORK = 'tiktok'

In [100]:
column_types = {'ID' : str}

# 1. Leitura de Arquivos e inicialização de Variáveis

Iniciação das variáveis que iremos utilizar e das colunas na qual iremos trabalhar


In [101]:
# 1. Pegar o modelo para testar
TYPE_MODEL = 'mixedbread-ai/mxbai-embed-large-v1'
model = SentenceTransformer(TYPE_MODEL)

# 2. Pegar as sentenças (nesse caso, no Post-filtrado)
file_pathL = BASE_PATH + 'input/LulaTotal Validado.xlsx'
file_pathB = BASE_PATH + 'input/Bolsonaro Validado.xlsx'
file_path_features = 'Embeddings_mxbai-embed-large-v1_'+ (SOCIAL_NETWORK) +'.xlsx'
#é bom modificar o nome manualmente, pois pode dar erro na questão do save

column_text = "Texto"
column_id = "ID"
column_author = "Perfil"
column_likes = "Curtidas"
column_inicial_date = "DataColeta"
column_final_date = "DataPost"
column_cand = "Candidato"

In [102]:
# converte para datetime 
def adjust_dates(df_aux):
    df_aux['DataColeta'] = pd.to_datetime(df_aux['DataColeta'])
    df_aux['DataPost'] = pd.to_datetime(df_aux['DataPost'])

    try:
        # converte esta coluna para conter uma string com o dia da semana
        df_aux['DiaDaSemana'] = pd.to_datetime(df_aux['DiaDaSemana'])
        df_aux['DiaDaSemana'] = df_aux['DiaDaSemana'].dt.strftime('%A')
    except:
        print("Coluna DiaDaSemana parece já estar guardado como date. Não precisa transformar!")

In [103]:
# 3. Ler os Arquivos tanto de Lula quanto de bolsonaro
df_lula = pd.read_excel(file_pathL, dtype=column_types)
df_bolsonaro = pd.read_excel(file_pathB, dtype=column_types)

df_lula.shape, df_bolsonaro.shape


((308, 22), (269, 22))

União das planilha e limpeza de variáveis que estão em branco, ou seja, não possuem utilidade.

In [104]:
df_total = pd.concat([df_lula, df_bolsonaro], axis=0)
df_total.shape

(577, 22)

In [105]:
len(df_total['ID'].unique())


577

In [106]:
df_total = df_total.reset_index(drop=True)
df_total.head(5)

,Unnamed: 0,DataColeta,Perfil,DataPost,DiaDaSemana,Plays,Curtidas,Comentarios,Compart.,Texto,...,LinkPost,ID,Duracao,Retórica Aristotélica,Dispositivo Retórico,Tipo de conteúdo,Abordagem,Tonalidade,Main character,Texto / Hashtag
0,1,2022-10-02,lulaoficial,2022-06-30 00:00:00,1900-01-05 00:00:00,196800.0,11700,809,589.0,Alô alô geração tiktoker! Imagina só um #gover...,...,https://www.tiktok.com/@lulaoficial/video/7115...,7115033431473474822,17.30,Pathos,Political Statement,Political-Purposeful,Acclamation,Neutral,Self alone,Texto + Hashtag
1,2,2022-10-02,lulaoficial,2022-06-30 00:00:00,1900-01-05 00:00:00,522000.0,33600,3324,3973.0,Já imaginou um #governo feito pra que as pesso...,...,https://www.tiktok.com/@lulaoficial/video/7115...,7115174031162215686,60.16,Pathos,Political Statement,Political-Purposeful,Acclamation,Neutral,Self alone,Texto + Hashtag
2,3,2022-10-02,lulaoficial,2022-07-01 00:00:00,1900-01-06 00:00:00,427900.0,34600,2289,1752.0,O pai ta estourado!😎 marque nos comentários um...,...,https://www.tiktok.com/@lulaoficial/video/7115...,7115357413712153861,14.88,Ethos,Humor,Campaign Act,Acclamation,Positive,Self alone,Texto + Hashtag
3,4,2022-10-02,lulaoficial,2022-07-01 00:00:00,1900-01-06 00:00:00,882200.0,47500,4312,2257.0,A gente tem um um encontro marcado no dia 02 d...,...,https://www.tiktok.com/@lulaoficial/video/7115...,7115560675824422149,15.39,Pathos,Call to Action,Campaign Act,Acclamation,Positive,Self alone,Texto + Hashtag
4,5,2022-10-02,lulaoficial,2022-07-02 00:00:00,1900-01-07 00:00:00,262200.0,22400,2150,1438.0,#PepsiApplePieChallenge Estamos fazendo uma #c...,...,https://www.tiktok.com/@lulaoficial/video/7115...,7115793869152734470,60.93,Pathos,Commitment,Political-Ideological,Acclamation,Positive,Self + voters,Texto + Hashtag


In [107]:
df_total.dropna(inplace=True)

Observar o que foi eliminado e conferir se está tudo certo:

In [108]:
df_total.shape

(568, 22)

In [109]:
df_total[df_total.isna().sum(axis=1) > 0].groupby('Perfil').count()

,Unnamed: 0,DataColeta,DataPost,DiaDaSemana,Plays,Curtidas,Comentarios,Compart.,Texto,LinkFoto,...,LinkPost,ID,Duracao,Retórica Aristotélica,Dispositivo Retórico,Tipo de conteúdo,Abordagem,Tonalidade,Main character,Texto / Hashtag
Perfil,,,,,,,,,,,,,,,,,,,,,


In [110]:
df_total.loc[df_total.isna().sum(axis=1) > 0].shape
df_total.shape

(568, 22)

# 2. Configuração da planilha pré-processada
## 2.1 Limpeza de colunas 

Drop de colunas que não iremos utilizar como por exemplo `Duração` ou até `Dispositivo Retórico` etc.

In [111]:
df_total.drop(columns=["Unnamed: 0","Plays","Comentarios","Compart.","Duracao", "LinkFoto", "LinkVideo", "LinkPost", 
                       "Retórica Aristotélica", "Dispositivo Retórico", "Tipo de conteúdo",
                  "Abordagem", "Tonalidade", "Main character", "Texto / Hashtag"], inplace=True)
df_total.tail(5)

,DataColeta,Perfil,DataPost,DiaDaSemana,Curtidas,Texto,ID
572,2022-10-30,bolsonaromessiasjair,2022-10-29 09:14:06.702000,1900-01-07 00:00:00,95300,#bolsonaro #rock #brasil #good #vibes #jairbol...,7159766081593150726
573,2022-10-30,bolsonaromessiasjair,2022-10-29 09:14:01.295000,1900-01-07 00:00:00,240900,- Muito brigado a você que nos acompanhou até ...,7159773640030997766
574,2022-10-30,bolsonaromessiasjair,2022-10-29 09:13:55.570000,1900-01-07 00:00:00,109600,#bolsonaro #lula #empolgado #brasil #🇧🇷,7159891102143630597
575,2022-10-30,bolsonaromessiasjair,2022-10-29 12:13:51.619000,1900-01-07 00:00:00,52100,#vacina #presidente #jair #bolsonaro #comparti...,7159943895445441797
576,2022-10-30,bolsonaromessiasjair,2022-10-29 17:13:45.948000,1900-01-07 00:00:00,23500,- Belo Horizonte/MG. - Presidente Jair Bolsona...,7160017587160485125


In [112]:
df_total.loc[df_total.isna().sum(axis=1) > 0].shape
df_total.dropna(inplace=True)
df_total.shape

(568, 7)

Conversão de datas para calcular a coluna de `Dias Decorridos`:

In [113]:
# converte para datetime
adjust_dates(df_total)
df_total.tail(5)

,DataColeta,Perfil,DataPost,DiaDaSemana,Curtidas,Texto,ID
572,2022-10-30,bolsonaromessiasjair,2022-10-29 09:14:06.702,Sunday,95300,#bolsonaro #rock #brasil #good #vibes #jairbol...,7159766081593150726
573,2022-10-30,bolsonaromessiasjair,2022-10-29 09:14:01.295,Sunday,240900,- Muito brigado a você que nos acompanhou até ...,7159773640030997766
574,2022-10-30,bolsonaromessiasjair,2022-10-29 09:13:55.570,Sunday,109600,#bolsonaro #lula #empolgado #brasil #🇧🇷,7159891102143630597
575,2022-10-30,bolsonaromessiasjair,2022-10-29 12:13:51.619,Sunday,52100,#vacina #presidente #jair #bolsonaro #comparti...,7159943895445441797
576,2022-10-30,bolsonaromessiasjair,2022-10-29 17:13:45.948,Sunday,23500,- Belo Horizonte/MG. - Presidente Jair Bolsona...,7160017587160485125


In [114]:
df_total.groupby(['DataColeta']).count()

,Perfil,DataPost,DiaDaSemana,Curtidas,Texto,ID
DataColeta,,,,,,
2022-10-02,425,425,425,425,425,425
2022-10-30,143,143,143,143,143,143


In [115]:
df_total['DiasDecorridos'] = (df_total['DataColeta'] - df_total['DataPost']).dt.days
df_total

df_total['Candidato'] = df_total['Perfil'].apply(lambda x: 'Lula' if x == 'lulaoficial' else 'Bolsonaro')

## 2.2 Flag para análise de Hashtags

Função para verificar se o texto contém apenas hashtags

Caso o comentário só possua hashtags ele irá retornar  `true`, caso contrário a função retorna `false`

In [116]:
import re

def contains_only_hashtags(text):
    hashtags = re.findall(r'#\S+', text)
    return len(hashtags) == len(text.split())

In [117]:
ids = df_total[column_id].tolist()
authors = df_total[column_author].tolist()
sentences = df_total[column_text].tolist()
likes = df_total[column_likes].tolist()
days = df_total['DiasDecorridos'].tolist()
candidatos = df_total['Candidato'].tolist()

Encode dos embeddings utilizando a library do "sentence-transformers" e também a concatenação de dados previamente informados da planilha passada como `ID`,`Candidato` e `Curtidas`

In [118]:
# 4. Calcular os embeddings das sentenças
embeddings = model.encode(sentences)
df_embeddings = pd.DataFrame(embeddings) 
df_embeddings.columns = [f'x{i+1}' for i in range(df_embeddings.shape[1])]



In [119]:
df_embeddings.tail(5)

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x1015,x1016,x1017,x1018,x1019,x1020,x1021,x1022,x1023,x1024
563,-0.140963,0.067802,-0.523615,0.616634,0.650749,0.198259,1.257955,0.662783,0.548705,0.035463,...,1.438568,0.193121,-0.372893,-0.108698,0.829201,0.856454,0.416607,0.499899,-0.677160,-0.132298
564,0.042332,-0.234858,-0.814700,0.823340,0.276801,-0.184168,0.273145,0.466900,0.276868,0.076657,...,0.821360,-0.047211,-0.464524,-0.567385,0.530373,0.455084,0.080720,0.427314,-0.916609,-0.430164
565,-0.054606,0.226093,-0.670925,0.485061,0.293188,-0.113349,1.019229,0.767915,0.623170,-0.144878,...,1.286832,0.045715,-0.293765,-0.001618,0.457520,0.384483,0.181806,-0.229281,-0.835020,0.248701
566,-0.287731,-0.240716,-0.265117,0.954569,0.586743,-0.036566,0.132282,0.688178,-0.256241,-0.040061,...,0.839534,-0.199658,-0.418112,-0.586381,0.876077,0.427513,0.246161,0.348530,-0.410696,-0.290743
567,0.191896,-0.456486,-0.553989,0.453893,0.459033,-0.403800,0.345611,0.108222,-0.224890,-0.076186,...,0.400670,0.308563,-0.724864,-0.097887,0.583676,0.407835,0.063030,-0.345539,-1.278493,0.869324


A planilha ficará com as colunas importantes originais (__ID__,__Candidato__ e __Curtidas__), e também estará com uma flag que mostra se há hashtags(__Only Hashtags__), e por fim, features calculadas pelo embeddinG   __x1__,__x2__,__x3__ ... __xN__ , onde N seria o tamanho de dimensões que aquele modelo possui

# 3. Concatenar e Salvar o Arquivo

In [120]:
df_final = pd.DataFrame({
    column_id: ids,
    'Candidato': candidatos,
    column_likes: likes,
    'Dias Decorridos': days
    
})

df_final['Only Hashtags'] = df_total[column_text].apply(contains_only_hashtags)
df_final['Only Hashtags'] = df_final['Only Hashtags'].fillna(False)  # Para lidar com tabulações que o pandas não reconhece

# Concatena com df_embeddings
df_final = pd.concat([df_final, df_embeddings], axis=1)


C:\Users\Kabum\AppData\Local\Temp\ipykernel_10392\194722803.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_final['Only Hashtags'] = df_final['Only Hashtags'].fillna(False)  # Para lidar com tabulações que o pandas não reconhece


In [121]:
df_final.tail(5)


,ID,Candidato,Curtidas,Dias Decorridos,Only Hashtags,x1,x2,x3,x4,x5,...,x1015,x1016,x1017,x1018,x1019,x1020,x1021,x1022,x1023,x1024
563,7159766081593150726,Bolsonaro,95300,0,True,-0.140963,0.067802,-0.523615,0.616634,0.650749,...,1.438568,0.193121,-0.372893,-0.108698,0.829201,0.856454,0.416607,0.499899,-0.677160,-0.132298
564,7159773640030997766,Bolsonaro,240900,0,True,0.042332,-0.234858,-0.814700,0.823340,0.276801,...,0.821360,-0.047211,-0.464524,-0.567385,0.530373,0.455084,0.080720,0.427314,-0.916609,-0.430164
565,7159891102143630597,Bolsonaro,109600,0,True,-0.054606,0.226093,-0.670925,0.485061,0.293188,...,1.286832,0.045715,-0.293765,-0.001618,0.457520,0.384483,0.181806,-0.229281,-0.835020,0.248701
566,7159943895445441797,Bolsonaro,52100,0,True,-0.287731,-0.240716,-0.265117,0.954569,0.586743,...,0.839534,-0.199658,-0.418112,-0.586381,0.876077,0.427513,0.246161,0.348530,-0.410696,-0.290743
567,7160017587160485125,Bolsonaro,23500,0,True,0.191896,-0.456486,-0.553989,0.453893,0.459033,...,0.400670,0.308563,-0.724864,-0.097887,0.583676,0.407835,0.063030,-0.345539,-1.278493,0.869324


## 3.1 Salvar o arquivo

In [122]:

df_final.to_excel((DIRECTORY + file_path_features), index=False)

print("Arquivo salvo:", file_path_features)

Arquivo salvo: Embeddings_mxbai-embed-large-v1_tiktok.xlsx
